In [1]:
import pandas as pd
import glob
import os
#os.getcwd()
os.chdir("/home/matthew/anaconda3/envs/insight/staycationLA/data/raw")

'/home/matthew/anaconda3/envs/insight/staycationLA/notebooks'

In [2]:
#os.chdir('/home/matthew/anaconda3/envs/insight/staycationLA/notebooks')
#os.getcwd()

In [15]:
LA = pd.read_csv("../data/processed/LA_combined_reviews.csv", index_col=None)
pos=LA['stars'] > 35
pos_LA=LA[pos]
pos_LA2=pos_LA.drop(['title', 'text'], axis=1)
pos_LA2

,stars,user,attraction
28793,40,Y3360JRanneh,The_Getty_Center
28794,40,grandslamhomer,Ahmanson_Theatre
28795,40,grandslamhomer,Airport_in_the_Sky
28796,40,luvSweetz,Hollywood_Walk_of_Fame
28797,40,grandslamhomer,Catalina_Island_Casino
...,...,...,...
156485,50,EmiratiVacationer,Rodeo_Drive
156486,50,alfredb971,Off_Track_Betting_at_Hollywood_Park
156487,50,EmiratiVacationer,Third_Street_Promenade
156488,50,jeanhV5194YE,Santa_Monica_Pier


In [16]:
is_NaN = pos_LA2. isnull()
row_has_NaN = is_NaN. any(axis=1)
rows_with_NaN = pos_LA2[row_has_NaN]
rows_with_NaN
pos_LA3=pos_LA2.dropna()

In [5]:
extension = 'csv'
all_filenames = [i for i in glob.glob('user_LA_*.{}'.format(extension))]
#print(all_filenames)

In [6]:
#combine all files in the list
li = []

for filename in all_filenames:
    df = pd.read_csv(filename, index_col=None)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame2=frame.dropna(axis=0, how='any') #remove duplicate rows (just in case)
frame3=frame2[~frame2.place.str.contains(", USA")] #remove all domestic rows
frame4=frame3[~frame2.place.str.contains(", Canada")] 
#export to csv
frame4.to_csv( "../processed/LA_combined_user_travel_hist.csv", index=False, encoding='utf-8-sig')

<ipython-input-6-b4f6eb9c53fb>:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  frame4=frame3[~frame2.place.str.contains(", Canada")]


In [7]:
os.chdir('/home/matthew/anaconda3/envs/insight/staycationLA/notebooks')
trav_hist = pd.read_csv("../data/processed/LA_combined_user_travel_hist.csv", index_col=None)
trav_hist

,tag,place,user
0,"""been""","Macau, China",Eddieg17467
1,"""been""","Playa de las Americas, Spain",Eddieg17467
2,"""been""","Kos Town, Greece",Eddieg17467
3,"""been""","Dublin, Ireland",Eddieg17467
4,"""been""","Cork, Ireland",Eddieg17467
...,...,...,...
2321947,"""been""","Hong Kong, China",buzz101
2321948,"""been""","Dubai, United Arab Emirates",buzz101
2321949,"""been""","St. Moritz, Switzerland",buzz101
2321950,"""been""","Nuevo Vallarta, Mexico",buzz101


In [11]:
city=trav_hist[trav_hist['place'].str.contains('Istanbul')]
city_users = city['user'].tolist()
len(city_users)

7941

In [12]:
city=trav_hist[trav_hist['place'].str.contains('Phuket')]
city_users = city['user'].tolist()
len(city_users)

2549

In [13]:
city=trav_hist[trav_hist['place'].str.contains('Jaipur')]
city_users = city['user'].tolist()
len(city_users)

1990

In [17]:
li = []

for user in city_users:
    test=pos_LA3[pos_LA3.user.str.contains(user)] 
    li.append(test)
    
frame = pd.concat(li, axis=0, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
frame2=frame.drop(['stars'], axis=1)
frame3=frame2.groupby('attraction').count()
frame4=frame3.sort_values(by='user', ascending=False)
frame4.head(10)

In [24]:
LA2=LA.drop(['stars', 'title', 'text'], axis=1)
LA3=LA2.dropna()

In [25]:
len(LA2)

156490

In [26]:
len(LA3)

155990

In [27]:
LA4=LA3.groupby('attraction').count()
LA4

,user
attraction,
24th_Street_Theatre,1
26_Mile_Bike_Path,978
6_Star_Villa_Foot_Spa,1
777_Tower,1
849_Building,1
...,...
Zimmer_Children_s_Museum,16
Zombie_Joe_s_Underground_Theatre,4
Zuma_Beach,498


In [28]:
merged_inner = pd.merge(left=LA4, right=frame4, left_on='attraction', right_on='attraction')
# In this case `species_id` is the only column name in  both dataframes, so if we skipped `left_on`
# And `right_on` arguments we would still get the same result

# What's the size of the output data?
#merged_inner.shape
merged_inner

,user_x,user_y
attraction,,
26_Mile_Bike_Path,978,124
A_Noise_Within_Theatre,26,5
Abalone_Cove_Shoreline_Park_Ecological_Reserve,81,9
Abbot_Kinney_Boulevard,274,35
Adamson_House_and_Malibu_Lagoon_Museum,153,19
...,...,...
Wrigley_Gardens,29,4
Wrigley_Memorial_Botanic_Garden,495,28
YouRaceLA,16,1


In [30]:
merged_inner['pop_control']= merged_inner['user_y']/merged_inner['user_x']
merged_inner2=merged_inner.sort_values(by='pop_control', ascending=False)
merged_inner2.head(10)

,user_x,user_y,pop_control
attraction,,,
Wiretap_Brewing,1,2,2.0
Lopez_Adobe,1,1,1.0
Freak_s_Antiques,1,1,1.0
Santa_Clarita_Valley_Equestrian_Center,1,1,1.0
Secret_Rose_Theatre,1,1,1.0
San_Pedro_Theatre_Club,1,1,1.0
Hellman_Park,1,1,1.0
Corita_Art_Center,1,1,1.0
First_Congregational_Church_of_Los_Angeles,1,1,1.0


In [36]:
import numpy as np
merged_inner['log']= np.log((merged_inner['user_x'])+1)
merged_inner['log2']= np.log((merged_inner['log'])+1)

In [39]:
merged_inner['pop_control2']= merged_inner['pop_control']*merged_inner['log']
merged_inner2=merged_inner.sort_values(by='pop_control2', ascending=False)
merged_inner2.head(20)

,user_x,user_y,pop_control,log,pop_control2,log2
attraction,,,,,,
Virginia_Robinson_Gardens,22,11,0.500000,3.135494,1.567747,1.419607
Route_66_End_of_the_Trail_Sign,5,4,0.800000,1.791759,1.433408,1.026672
Wiretap_Brewing,1,2,2.000000,0.693147,1.386294,0.526589
Odyssey_Theatre,6,4,0.666667,1.945910,1.297273,1.080418
Lummis_Home_El_Alisal,10,5,0.500000,2.397895,1.198948,1.223156
Beverly_Hills_Public_Library,67,18,0.268657,4.219508,1.133599,1.652403
USS_Los_Angeles_Memorial,2,2,1.000000,1.098612,1.098612,0.741276
Ohana_Brewing_Company,5,3,0.600000,1.791759,1.075056,1.026672
Norton_Simon_Museum,1071,162,0.151261,6.977281,1.055387,2.076598


In [40]:
merged_inner['pop_control2']= merged_inner['pop_control']*merged_inner['log2']
merged_inner2=merged_inner.sort_values(by='pop_control2', ascending=False)
merged_inner2.head(20)

,user_x,user_y,pop_control,log,pop_control2,log2
attraction,,,,,,
Wiretap_Brewing,1,2,2.000000,0.693147,1.053178,0.526589
Route_66_End_of_the_Trail_Sign,5,4,0.800000,1.791759,0.821338,1.026672
USS_Los_Angeles_Memorial,2,2,1.000000,1.098612,0.741276,0.741276
Odyssey_Theatre,6,4,0.666667,1.945910,0.720279,1.080418
Virginia_Robinson_Gardens,22,11,0.500000,3.135494,0.709803,1.419607
Ohana_Brewing_Company,5,3,0.600000,1.791759,0.616003,1.026672
Lummis_Home_El_Alisal,10,5,0.500000,2.397895,0.611578,1.223156
Leimert_Park_Village,3,2,0.666667,1.386294,0.579828,0.869742
South_Robertson_Boulevard,3,2,0.666667,1.386294,0.579828,0.869742


In [ ]:
LA = pd.read_csv("../data/processed/LA_final_corpora.csv", index_col=None)

In [6]:
pd.options.display.max_colwidth = 20
LA.head(10)

,attraction,english,TA_url,rating,gps,url,type,name,pic,num_reviews,ranking
0,Universal_Studios_Hollywood,front of the line be a must family have be to ...,https://www.tripadvisor.com/Attraction_Review-...,45,"[('34.138416', '-118.35337')]",http://www.universalstudioshollywood.com/,Theme Parks,Universal Studios Hollywood,https://dynamic-media-cdn.tripadvisor.com/medi...,36470,1
1,Griffith_Observatory,observatory another must see landmark visit to...,https://www.tripadvisor.com/Attraction_Review-...,45,"[('34.118492', '-118.30039')]",http://www.griffithobservatory.org/,Observatories & Planetariums,Griffith Observatory,https://dynamic-media-cdn.tripadvisor.com/medi...,20247,2
2,Santa_Monica_Pier,the end of the mother road a must do on any tr...,https://www.tripadvisor.com/Attraction_Review-...,40,"[('34.01038', '-118.49594')]",http://santamonicapier.org/,Points of Interest & Landmarks,Santa Monica Pier,https://dynamic-media-cdn.tripadvisor.com/medi...,18866,3
3,Hollywood_Walk_of_Fame,exciting be very crowded walk up down the stre...,https://www.tripadvisor.com/Attraction_Review-...,35,"[('34.1016', '-118.32672')]",http://www.walkoffame.com/,Points of Interest & Landmarks,Hollywood Walk of Fame,https://dynamic-media-cdn.tripadvisor.com/medi...,17542,4
4,The_Getty_Center,accidentally press wrong key but can t get out...,https://www.tripadvisor.com/Attraction_Review-...,50,"[('34.077564', '-118.474625')]",http://www.getty.edu/visit/center,Specialty Museums,The Getty Center,https://dynamic-media-cdn.tripadvisor.com/medi...,14503,5
5,Venice_Beach,crazy i think be crazy in a good way from the ...,https://www.tripadvisor.com/Attraction_Review-...,40,"[('33.985912', '-118.47328')]",https://www.tripadvisor.com/Attraction_Review-...,Beaches,Venice Beach,https://dynamic-media-cdn.tripadvisor.com/medi...,10729,6
6,Hollywood_Sign,beautiful landmark definitely a must if be vis...,https://www.tripadvisor.com/Attraction_Review-...,40,"[('34.134113', '-118.32154')]",http://www.visitcalifornia.com/attraction/holl...,Points of Interest & Landmarks,Hollywood Sign,https://dynamic-media-cdn.tripadvisor.com/medi...,7648,7
7,Santa_Monica_State_Beach,beach a beach have long shore with a broad wal...,https://www.tripadvisor.com/Attraction_Review-...,45,"[('34.0114', '-118.49491')]",http://www.smgov.net/portals/beach,State Parks,Santa Monica State Beach,https://dynamic-media-cdn.tripadvisor.com/medi...,6697,8
8,Third_Street_Promenade,a s dream with atmosphere can not make a trip ...,https://www.tripadvisor.com/Attraction_Review-...,45,"[('34.01833', '-118.4993')]",http://www.downtownsm.com/,Shopping Malls,Third Street Promenade,https://dynamic-media-cdn.tripadvisor.com/medi...,6098,9
9,Rodeo_Drive,pretty shop super exclusive and luxury shoppin...,https://www.tripadvisor.com/Attraction_Review-...,40,"[('34.06983', '-118.40366')]",http://www.beverlyhills.org/exploring/beverlyh...,Points of Interest & Landmarks,Rodeo Drive,https://dynamic-media-cdn.tripadvisor.com/medi...,5491,10


In [ ]:
#section on star rating- doesn't work very well

In [312]:
reviews['stars'] / reviews.groupby('attraction')['stars'].transform('sum')

0        0.026178
1        0.026178
2        0.026178
3        0.026178
4        0.026178
           ...   
62399    0.000395
62400    0.000158
62401    0.000158
62402    0.000395
62403    0.000316
Name: stars, Length: 62404, dtype: float64

In [340]:
reviews.groupby('attraction')['stars'].transform('sum')

0          1910
1          1910
2          1910
3          1910
4          1910
          ...  
62399    126430
62400    126430
62401    126430
62402    126430
62403    126430
Name: stars, Length: 62404, dtype: int64

In [417]:
c = reviews.groupby('attraction')['stars'].sum().rename("count")
total=c.groupby(['attraction']).sum()
maximum=(reviews.groupby('attraction')['stars'].count())*50
av=total/maximum
import numpy as np
r4=np.log(reviews3+1)
r5=np.log(r4+1)
r5a=np.log(r5+1)
r5b=np.log(r5a+1)
r5c=np.log(r5b+1)
r5d=np.log(r5c+1)
r6=np.log(r5d+1)
r6['rating']=av
r6['score']=r6['user'] * r6['rating']
r7=r6.sort_values(by='score', ascending=False)
r7.head(10)

,user,rating,score
attraction,,,
LA_The_Huntington_Library_Art_Museum_and_Botanical_Gardens_url,0.371879,0.963305,0.358233
LA_Norton_Simon_Museum,0.368907,0.959551,0.353985
LA_The_Getty_Center_url,0.368513,0.955722,0.352197
LA_The_Huntington_Desert_Garden_url,0.356654,0.977778,0.348729
LA_Glass_Church_Wayfarers_Chapel_url,0.362504,0.961798,0.348656
LA_26_Mile_Bike_Path,0.369131,0.941138,0.347404
LA_The_Gamble_House,0.365217,0.943717,0.344662
LA_Frederick_R_Weisman_Art_Foundation,0.351903,0.977011,0.343813
LA_Southern_California_Soaring_Academy_url,0.349108,0.984615,0.343737


In [414]:
c = frame.groupby('attraction')['stars'].sum().rename("count")
total=c.groupby(['attraction']).sum()
maximum=(frame.groupby('attraction')['stars'].count())*50
av=total/maximum
import numpy as np
r4=np.log(frame3+1)
r5=np.log(r4+1)
r5a=np.log(r5+1)
r5b=np.log(r5a+1)
r5c=np.log(r5b+1)
r5d=np.log(r5c+1)
r6=np.log(r5d+1)
#r6=r4
r6['rating']=av
r6['score']=r6['user'] * r6['rating']
ist_r7=r6.sort_values(by='score', ascending=False)
ist_r7.head(10)

,user,rating,score
attraction,,,
LA_The_Huntington_Library_Art_Museum_and_Botanical_Gardens_url,0.346353,0.988000,0.342197
LA_Norton_Simon_Museum,0.347573,0.967857,0.336401
LA_Santa_Monica_State_Beach,0.349982,0.935211,0.327307
LA_26_Mile_Bike_Path,0.342181,0.954286,0.326539
LA_Aquarium_of_the_Pacific,0.348952,0.931250,0.324961
LA_Santa_Monica_Public_Library,0.329674,0.973333,0.320883
LA_Rodeo_Drive,0.346780,0.919231,0.318771
LA_Naples_Island,0.342181,0.931429,0.318718
LA_El_Dorado_Nature_Center,0.332738,0.955556,0.317950


In [413]:
c = frame.groupby('attraction')['stars'].sum().rename("count")
total=c.groupby(['attraction']).sum()
maximum=(frame.groupby('attraction')['stars'].count())*50
av=total/maximum
import numpy as np
r4=np.log(frame3+1)
r5=np.log(r4+1)
r5a=np.log(r5+1)
r6=np.log(r5+1)
#r6=r4
r6['rating']=av
r6['score']=r6['user'] * r6['rating']
ist_r7=r6.sort_values(by='score', ascending=False)
ist_r7.head(10)

,user,rating,score
attraction,,,
LA_The_Huntington_Library_Art_Museum_and_Botanical_Gardens_url,0.953860,0.988000,0.942413
LA_Norton_Simon_Museum,0.962415,0.967857,0.931480
LA_The_Queen_Mary,1.037628,0.883146,0.916377
LA_Santa_Monica_State_Beach,0.979564,0.935211,0.916099
LA_Third_Street_Promenade,1.026908,0.891892,0.915890
LA_Aquarium_of_the_Pacific,0.972187,0.931250,0.905349
LA_26_Mile_Bike_Path,0.925237,0.954286,0.882941
LA_Rodeo_Drive,0.956848,0.919231,0.879565
LA_Naples_Island,0.925237,0.931429,0.861793


In [415]:
r6.sort_values(by='user', ascending=False)

,user,rating,score
attraction,,,
LA_The_Queen_Mary,0.357802,0.883146,0.315991
LA_Third_Street_Promenade,0.356396,0.891892,0.317867
LA_Santa_Monica_State_Beach,0.349982,0.935211,0.327307
LA_Aquarium_of_the_Pacific,0.348952,0.931250,0.324961
LA_Norton_Simon_Museum,0.347573,0.967857,0.336401
...,...,...,...
LA_Industry_Hills_Golf_Club_url,0.234307,0.800000,0.187446
LA_International_City_Theatre,0.234307,1.000000,0.234307
LA_San_Antonio_Winery_url,0.234307,1.000000,0.234307


In [395]:
frame4.head(10)

,user
attraction,
LA_The_Queen_Mary,178
LA_Third_Street_Promenade,148
LA_Santa_Monica_State_Beach,71
LA_Aquarium_of_the_Pacific,64
LA_Norton_Simon_Museum,56
LA_Rodeo_Drive,52
LA_The_Huntington_Library_Art_Museum_and_Botanical_Gardens_url,50
LA_26_Mile_Bike_Path,35
LA_Naples_Island,35


In [224]:
l2[l2.user.str.contains("princesslollipop")] 

,stars,title,text,user,review_category
10,50,Cold but lovely,What a wonderful place to spend a few hours wi...,princesslollipop,Positive
